In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
# Import Libraries

# DS 
import os
from itertools import islice
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
import seaborn as sns

# PLOTLY
import plotly.offline as po
import plotly.graph_objs as go

# SKLEARN
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (GradientBoostingRegressor, 
                              GradientBoostingClassifier, 
                              AdaBoostClassifier,
                              AdaBoostRegressor,
                              RandomForestRegressor,
                              RandomForestClassifier)
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.svm import SVC
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence

# Aesthetic Plots 
%matplotlib inline

In [17]:
# Modules


In [18]:
# Helper Functions
def accuracy(model, X, y) -> np.float:
    y_pred = model.predict(X)
    return sum(y_pred==y) / len(y_pred)

def get_score(model, X, y) -> tuple:
    mse = np.mean(cross_val_score(model, X, y, scoring=make_scorer(mean_squared_error)))
    r2 = np.mean(cross_val_score(model, X, y, scoring=make_scorer(r2_score)))
    acc = np.mean(cross_val_score(model, X, y, scoring=make_scorer(accuracy)))
    print(f"""{model.__class__.__name__}     Train CV | MSE: {mse} | R2: {r2} | Acc: {acc}""")
    return mse, r2


# Create Basline Models

In [6]:
# Read and Clean CSV
churn = pd.read_csv("data/churn.csv")

In [ ]:
# Create X, y arrays from dataframe
X = churn
y = churn.pop("target")

In [8]:
# Train Test Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y)

NameError: name 'X' is not defined

## Random Forest Classifier

In [13]:
# Create Random Forest Model
model_rf = RandomForestClassifier(
                            oob_score=True,
                            max_features=3,
    
)

In [16]:
model_rf.__class__.__name__

'RandomForestClassifier'

In [ ]:
# Fit Data
model_rf.fit(X_train, y_train)